In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType

# Načtení a rozdělení datasetu na trénink a evaluaci modelu
dataset = load_dataset("json", data_files="everything_finetunning.json", field='train') #, field='train'
train_test = dataset["train"].train_test_split(test_size=0.1)  
train_dataset = train_test["train"]
eval_dataset = train_test["test"]

# Načtení modelu deepseek a tokenizeru (potřebné pro přeměnu čistého textu na tokeny, kterým model rozumí
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

# Nastavení LoRA (r = vliv fine-tuningu na vybrane parametry, lora_alpha = škálování finálního výstupu, lora_dropout = proti přeučení modelu, task_type = typ fine-tuningu
lora_config = LoraConfig(
    r=8,  # nebo 16
    lora_alpha=32,  # nebo 64
    lora_dropout=0.1,  
    task_type=TaskType.CAUSAL_LM  
)

# načtení modelu pro fine-tunining
model = get_peft_model(model, lora_config)

# Tokenizační funkce pro změnu textu na tokeny
def tokenize_function(examples):
    
    inputs = tokenizer(examples["prompt"], examples["response"], truncation=True, padding="max_length", max_length=1024)
    inputs["labels"] = inputs["input_ids"].copy()  
    return inputs

# Převod trénovacích dat na tokeny pomocí tokenizační funkce
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

# Parametry trénování. Parametry s vlivem na výstup jsou pospsány přímo v bakalařské práci.
training_args = TrainingArguments(
    output_dir="./results",  
    num_train_epochs=17,  
    per_device_train_batch_size=1,  
    per_device_eval_batch_size=1,  
    gradient_accumulation_steps=1,  
    save_strategy="epoch",  
    logging_dir="./logs", 
    logging_steps=10, 
    optim="adamw_bnb_8bit",  
    save_total_limit=2,  
    report_to="none",  
    evaluation_strategy="epoch",  
    learning_rate=5e-5,  
    weight_decay=0.01, 
    load_best_model_at_end=True,  
    metric_for_best_model="eval_loss",  
)

# Definice modulu Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer
)

# Inicializeca trénování
trainer.train()

# Uložení výsledků
trainer.save_model("./r8_mine_deepseek")

/opt/tljh/user/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
2025-04-17 13:01:23.212174: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 13:01:23.223948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744887683.236742 2221960 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

/opt/tljh/user/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_2221960/2078841516.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,0.666800,0.618648
2,0.505800,0.470173
3,0.402000,0.389869
4,0.412900,0.346089
5,0.324400,0.307330
6,0.280000,0.283239
7,0.274000,0.271806
8,0.213800,0.266345
9,0.256900,0.256566
10,0.186800,0.254705
